# Testing, Profiling, Concurrency Patterns
Objectives:
- Property-based tests (hypothesis) stub
- Mutation testing mention
- CPU vs memory profiling on a toy service
- Async vs threads vs processes for I/O
- Structured cancellation and timeouts

In [ ]:
# Starter imports
import asyncio
import time
from pathlib import Path

# TODO: add tests, profiling snippets, and concurrency comparisons

In [ ]:
# Property-based testing stub (pip install hypothesis)

try:
    from hypothesis import given, strategies as st
except ImportError:
    given = None
    st = None


def reverse_twice(xs):
    return list(reversed(list(reversed(xs))))


if given:
    @given(st.lists(st.integers()))
    def test_reverse_twice_is_identity(xs):
        assert reverse_twice(xs) == xs
else:
    print("Install hypothesis to run property tests: pip install hypothesis")

In [ ]:
import tracemalloc, time, math

def busy_work(n=200_000):
    return sum(math.sqrt(i) for i in range(n))

tracemalloc.start()
t0 = time.perf_counter()
busy_work(100_000)
elapsed = time.perf_counter() - t0
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

{
    "elapsed_s": round(elapsed, 4),
    "current_kb": round(current/1024, 1),
    "peak_kb": round(peak/1024, 1),
}

In [ ]:
import asyncio, concurrent.futures, time

async def io_task(delay=0.2):
    await asyncio.sleep(delay)
    return delay

def thread_task(delay=0.2):
    time.sleep(delay)
    return delay

def process_task(delay=0.2):
    time.sleep(delay)
    return delay

async def compare_concurrency(n=10):
    t0 = time.perf_counter()
    await asyncio.gather(*(io_task() for _ in range(n)))
    async_elapsed = time.perf_counter() - t0

    with concurrent.futures.ThreadPoolExecutor() as pool:
        t0 = time.perf_counter()
        list(pool.map(thread_task, [0.2]*n))
        thread_elapsed = time.perf_counter() - t0

    with concurrent.futures.ProcessPoolExecutor() as pool:
        t0 = time.perf_counter()
        list(pool.map(process_task, [0.2]*n))
        process_elapsed = time.perf_counter() - t0

    return {"asyncio": async_elapsed, "threads": thread_elapsed, "processes": process_elapsed}

await compare_concurrency(8)

In [ ]:
async def cancellable(delay=1.0):
    try:
        await asyncio.sleep(delay)
        return "done"
    except asyncio.CancelledError:
        print("cancelled")
        raise

async def demo_timeout():
    try:
        return await asyncio.wait_for(cancellable(2.0), timeout=0.5)
    except asyncio.TimeoutError:
        return "timed out"

await demo_timeout()

Notes:
- Mutation testing: try `mutmut` or `mutatest` to ensure tests kill mutants.
- Profile CPU with `cProfile`/`pyinstrument`; memory with `tracemalloc` as shown.
- Async shines for I/O-bound tasks; processes for CPU-bound; threads for C-extensions releasing the GIL.